In [ ]:
%run master_functions.ipynb

In [ ]:
res = 'MET'

ihmm = file_rw.load_object('saved_parameters/2000iter_%s_unseeded.pl' % res)['ihmm']

In [ ]:
A, sigma, mu, T, mu_weights = organize_parameters(ihmm) # arrange parameters into arrays suitable for clustering

# file_rw.save_object({'A': A, 'sigma': sigma, 'mu': mu, 'T': T}, 'organized_params.pl')

tot_clusters = 30  # total clusters if combine_clusters = True

# clusters on each variable if combine_clusters = False
nclusters_A = 3
nclusters_sigma = 3
nclusters_r = 2
nclusters_T = 4

# total clusters on non-T variables if cluster_T_separate = True
nonT_clusters = 10

combine_clusters = True
cluster_T_separate = False
cluster_vars = 'eigs'
algorithm = 'agglomerative'

# default is diags
eigs = False
diags = True
if cluster_vars == 'eigs':
    eigs = True
    diags = False

In [ ]:
nboot = 50  # number of bootstrap trials
frac = 0.4
ntraj = 100
dt = 0.5
endshow=2000
linkage='ward'  # possibilities: {“ward”, “complete”, “average”, “single”}
load = False
save = True

tot_clusters = [4, 6, 8, 10, 12, 14, 15, 16, 18, 20, 22, 24, 25, 26, 28, 30]
tot_clusters = [10]
mur = np.linalg.norm(mu[:2, :], axis=0)
if load:
    
    final_parameters, trajectory_generators, clustered_labels = file_rw.load_object('clusters_%s_%s.pl' %(res, linkage))
    print('Loaded clusters_%s_%s.pl' % (res, linkage))
else:
    
    trajectory_generators = {}
    final_parameters = {}
    clustered_labels = {}

    for i, nC in enumerate(tot_clusters):

        sig_cluster, all_state_params = cluster_parameters(sigma, A, mu, T, combine_clusters, cluster_T_separate, 
                                                           eigs, diags, nC, nonT_clusters, nclusters_T,
                                                           nclusters_sigma, nclusters_r, nclusters_A, linkage=linkage)
        
        clustered_labels[nC] = sig_cluster.labels
        final_parameters[nC] = parameterize_clusters(sig_cluster, ihmm, mu_weights, show=False)
        
        nsteps = final_parameters[nC]['z'].shape[1]

        trajectory_generators[nC] = GenARData(params=final_parameters[nC])
        trajectory_generators[nC].gen_trajectory(nsteps, ntraj, bound_dimensions=[0, 1], resample_T=True)

    if save:
        
        file_rw.save_object((final_parameters, trajectory_generators, clustered_labels), 'clusters_%s_%s.pl' %(res, linkage))

## Visualize clustering on individual variables

In [ ]:
hist_limits = [[None, None, None, None], [None, None, [0, 10], None], [None, None, [0, 200], None]]
axis_limits = [[[0.5, 1], None],
               [[0, 10], None],
               [[0, 200], None]]

visualize_clusters(T, 'T', nclusters_T, eigs, diags, nbins=25, algorithm='agglomerative', linkage='ward', hist_limits=hist_limits, axis_limits=axis_limits)

In [ ]:
params = {'sigma': sigma, 'A': A, 'mu': np.square(np.linalg.norm(mu[:2, :], axis=0)), 'T': -np.log(1 - T)}
silhouette_(params, eigs, diags, 'ward', np.arange(2, 30), algorithm='agglomerative', plot=True, 
            savename='/home/ben/github/LLC_Membranes/Ben_Manuscripts/hdphmm/supporting_figures/silhouette_%s.pdf' % res)